In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('statlog.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,1
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,1
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,1
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,1


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 0, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,0
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,0
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,0
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,0


In [5]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [6]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [7]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [8]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [9]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [10]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 2s 126ms/step - loss: 0.8075 - accuracy: 0.5602 - val_loss: 0.6620 - val_accuracy: 0.6481
Epoch 2/100
5/5 [==============================] - 0s 33ms/step - loss: 0.5811 - accuracy: 0.7176 - val_loss: 0.6402 - val_accuracy: 0.6296
Epoch 3/100
5/5 [==============================] - 0s 37ms/step - loss: 0.5954 - accuracy: 0.7269 - val_loss: 0.6300 - val_accuracy: 0.6296
Epoch 4/100
5/5 [==============================] - 0s 33ms/step - loss: 0.4377 - accuracy: 0.7731 - val_loss: 0.6257 - val_accuracy: 0.6296
Epoch 5/100
5/5 [==============================] - 0s 33ms/step - loss: 0.4877 - accuracy: 0.7778 - val_loss: 0.6257 - val_accuracy: 0.6296
Epoch 6/100
5/5 [==============================] - 0s 35ms/step - loss: 0.4993 - accuracy: 0.7824 - val_loss: 0.6255 - val_accuracy: 0.6667
Epoch 7/100
5/5 [==============================] - 0s 17ms/step - loss: 0.4087 - accuracy: 0.8194 - val_loss: 0.6255 - val_accuracy: 0.6667
Epoch 8/100
5/5 [==

5/5 [==============================] - 0s 18ms/step - loss: 0.1518 - accuracy: 0.9491 - val_loss: 0.4612 - val_accuracy: 0.7963
Epoch 60/100
5/5 [==============================] - 0s 18ms/step - loss: 0.1277 - accuracy: 0.9537 - val_loss: 0.4750 - val_accuracy: 0.7778
Epoch 61/100
5/5 [==============================] - 0s 15ms/step - loss: 0.1381 - accuracy: 0.9537 - val_loss: 0.4770 - val_accuracy: 0.7778
Epoch 62/100
5/5 [==============================] - 0s 16ms/step - loss: 0.1261 - accuracy: 0.9630 - val_loss: 0.4721 - val_accuracy: 0.8148
Epoch 63/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0993 - accuracy: 0.9630 - val_loss: 0.4720 - val_accuracy: 0.8333
Epoch 64/100
5/5 [==============================] - 0s 20ms/step - loss: 0.0951 - accuracy: 0.9722 - val_loss: 0.4612 - val_accuracy: 0.8148
Epoch 65/100
5/5 [==============================] - 0s 33ms/step - loss: 0.0913 - accuracy: 0.9676 - val_loss: 0.4506 - val_accuracy: 0.8333
Epoch 66/100
5/5 [========

In [11]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 2ms/step - loss: 0.4385 - accuracy: 0.8333
Test Loss: 0.4385
Test Accuracy: 0.8333


In [12]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 0s 0s/step
Confusion Matrix:
[[29  4]
 [ 5 16]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.87        33
           1       0.80      0.76      0.78        21

    accuracy                           0.83        54
   macro avg       0.83      0.82      0.82        54
weighted avg       0.83      0.83      0.83        54



In [13]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [14]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [15]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 13s 662ms/step - loss: 0.6731 - accuracy: 0.5602 - val_loss: 0.6866 - val_accuracy: 0.5926
Epoch 2/100
5/5 [==============================] - 0s 68ms/step - loss: 0.6184 - accuracy: 0.7269 - val_loss: 0.6730 - val_accuracy: 0.6111
Epoch 3/100
5/5 [==============================] - 0s 68ms/step - loss: 0.5821 - accuracy: 0.7407 - val_loss: 0.6581 - val_accuracy: 0.7778
Epoch 4/100
5/5 [==============================] - 0s 70ms/step - loss: 0.5467 - accuracy: 0.7407 - val_loss: 0.6426 - val_accuracy: 0.7037
Epoch 5/100
5/5 [==============================] - 0s 70ms/step - loss: 0.5251 - accuracy: 0.7593 - val_loss: 0.6261 - val_accuracy: 0.6296
Epoch 6/100
5/5 [==============================] - 0s 73ms/step - loss: 0.5121 - accuracy: 0.7639 - val_loss: 0.6198 - val_accuracy: 0.6481
Epoch 7/100
5/5 [==============================] - 0s 45ms/step - loss: 0.5098 - accuracy: 0.7731 - val_loss: 0.6314 - val_accuracy: 0.6111
Epoch 8/100
5/5 [=

In [16]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 10ms/step - loss: 0.5690 - accuracy: 0.7407
Test Loss: 0.5690
Test Accuracy: 0.7407


In [17]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 2s 10ms/step
Confusion Matrix:
[[28  5]
 [ 9 12]]

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.85      0.80        33
           1       0.71      0.57      0.63        21

    accuracy                           0.74        54
   macro avg       0.73      0.71      0.72        54
weighted avg       0.74      0.74      0.73        54



In [18]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [19]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [20]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 19s 816ms/step - loss: 0.6581 - accuracy: 0.6296 - val_loss: 0.6940 - val_accuracy: 0.4444
Epoch 2/100
5/5 [==============================] - 0s 81ms/step - loss: 0.5404 - accuracy: 0.7824 - val_loss: 0.6962 - val_accuracy: 0.4259
Epoch 3/100
5/5 [==============================] - 0s 75ms/step - loss: 0.5090 - accuracy: 0.7824 - val_loss: 0.6976 - val_accuracy: 0.4074
Epoch 4/100
5/5 [==============================] - 0s 78ms/step - loss: 0.4526 - accuracy: 0.7917 - val_loss: 0.7027 - val_accuracy: 0.3889
Epoch 5/100
5/5 [==============================] - 0s 75ms/step - loss: 0.4537 - accuracy: 0.7963 - val_loss: 0.7090 - val_accuracy: 0.3889
Epoch 6/100
5/5 [==============================] - 0s 75ms/step - loss: 0.4565 - accuracy: 0.8241 - val_loss: 0.7159 - val_accuracy: 0.3889
Epoch 7/100
5/5 [==============================] - 0s 73ms/step - loss: 0.4338 - accuracy: 0.8241 - val_loss: 0.7232 - val_accuracy: 0.3889
Epoch 8/100
5/5 [=

In [21]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 20ms/step - loss: 0.6940 - accuracy: 0.4444
Test Loss: 0.6940
Test Accuracy: 0.4444


In [22]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 3s 20ms/step
Confusion Matrix:
[[ 3 30]
 [ 0 21]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.09      0.17        33
           1       0.41      1.00      0.58        21

    accuracy                           0.44        54
   macro avg       0.71      0.55      0.38        54
weighted avg       0.77      0.44      0.33        54



In [23]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



2/2 [==============================] - 0s 10ms/step


In [24]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [25]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7778


In [26]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[25  8]
 [ 4 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.76      0.81        33
           1       0.68      0.81      0.74        21

    accuracy                           0.78        54
   macro avg       0.77      0.78      0.77        54
weighted avg       0.79      0.78      0.78        54

Ensemble Accuracy: 0.7778


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [28]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [29]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [30]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [31]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [32]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8704


In [33]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[30  3]
 [ 4 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.90        33
           1       0.85      0.81      0.83        21

    accuracy                           0.87        54
   macro avg       0.87      0.86      0.86        54
weighted avg       0.87      0.87      0.87        54

Ensemble Accuracy: 0.8704


In [34]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [35]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8519
Confusion Matrix:
[[30  3]
 [ 5 16]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        33
           1       0.84      0.76      0.80        21

    accuracy                           0.85        54
   macro avg       0.85      0.84      0.84        54
weighted avg       0.85      0.85      0.85        54

Ensemble Accuracy: 0.8519
